# Measures sanity check

We plot the computed measures in 1D to see if they behave as we expect.

In [ ]:
import os
import sys
import itertools

# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats

import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

sys.path.append("/nfs/scistore12/chlgrp/vvolhejn/smooth")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%autoreload 1

In [ ]:
def saw(x):
    x = (x+1) % 2
    return np.min([x, 2-x], axis=0)

x_train = np.linspace(-2, 2, 5, dtype=np.float32).reshape((-1, 1))
y_train = saw(x_train)

x_test = np.linspace(-3, 3, 61, dtype=np.float32).reshape((-1, 1))
y_test = saw(x_test)

dataset = smooth.datasets.Dataset(x_train, y_train, x_test, y_test)

In [ ]:
def train_models(activation, n=10, plot=False):
    init_scales = np.logspace(np.log10(10.), np.log10(1000.), n)
    models = dict()

    for init_scale in init_scales:
        models[init_scale] = smooth.model.train_shallow(
            dataset,
            learning_rate=0.1 / init_scale,
            init_scale=init_scale,
            epochs=10000,
            hidden_size=100,
            batch_size=len(x_train),
            verbose=0,
            callbacks=[
                tf.keras.callbacks.EarlyStopping("loss", min_delta=1e-5, patience=100),
                smooth.callbacks.Tqdm(verbose=0),
                tf.keras.callbacks.TerminateOnNaN()
            ],
            train_val_split=1.0,
            activation=activation,
        )

        if plot:
            smooth.analysis.plot_shallow(models[init_scale], dataset)
        
    return models

In [ ]:
models_relu = train_models("relu", plot=True)

In [ ]:
models_tanh = train_models("tanh", plot=True)

In [ ]:
def plot_measures(models):
    measures = dict()

    for init_scale, model in tqdm.notebook.tqdm(models.items()):
        measures[init_scale] = smooth.measures.get_measures(model, x_test, y_test)

    measure_names = measures[init_scales[0]].keys()

    x = sorted(measures.keys())
    yd = dict(zip(measure_names,[[] for _ in range(len(measure_names))]))

    for init_scale in x:
        for k, v in measures[init_scale].items():
            yd[k].append(v)

    for measure_name in measure_names:
        plt.plot(x, yd[measure_name])
        plt.title(measure_name)
        plt.xscale("log")
        plt.show()
    
    return measures

In [ ]:
measures_relu = plot_measures(models_relu)

In [ ]:
measures_tanh = plot_measures(models_tanh)

## Sanity check

We are evaluating measures on the interval $[-3, 3]$. For "smooth" functions, we would expect:

- `gradient_norm` $ \approx 1$, because it approximates $\mathbb{E}(\lvert f'(x) \rvert)$
- `seg_total_variation` $ \approx 6$, because it approximates the total variation of $f$, that is $\int_{-3}^{3} \lvert f'(x) \rvert$
- `seg_total_variation_derivative` $ \approx 6$, because it approximates the total variation of $f'$ (not expressible as an integral if the function is not twice differentiable, as with ReLU nets

In [ ]:
measures_relu[10.0]

In [ ]:
measures_tanh[10.0]